# Regularización

Se recordamos, a la hora de entrenar redes neuronales podemos estar en el caso del subentrenamiento, que esté bien entrneada, o del sobreentrenamiento

![subentrenamiento, entrenamiento y sobreentrenamiento](../01%20Introduccion%20a%20las%20redes%20neuronales/Imagenes/sobreentrenamiento.png)

Pues bien, a la hora de resolver un problema con redes neuronales, para evitar estar en el primer caso, en el de subentrenamiento, lo que se suele hacer es usar redes grandes para resolver el problema. Probablemente redes más grandes de lo que en realidad necesitamos.

Esto hace que podamos caer en el último caso, en el del sobreentrenamiento. Por lo que paraevitar sobreentrenar se suelen usar métodos de regularización que evitan esto.

Vamos a ver algunos

## Regularización L2

Consiste en sumar un termino en la función ded coste

$$loss_{L2} = loss + \frac{\lambda}{2N}\sum_{i=1}^{N}{\omega_i^2}$$

$\lambda$ es un hiperparámetro que definimos nosotros y que indica la importancia o la magnitod de la regularización

¿Por qué funciona la regularización L2? Como hemos visto, durante el entrenamiento, los pesos de la red se actualizan restándoles el gradiente de la función de coste, multiplicada por el learning rate $\alpha$. Así que vamos ha calcular el gradiente

$$\frac{\partial loss_{L2}}{\partial \omega_i} = \frac{\partial loss}{\partial \omega_i} + \frac{\partial}{\partial \omega_i}\left[\frac{\lambda}{2N}\sum_{i=1}^{N}{\omega_i^2}\right] = $$
$$ = \frac{\partial loss}{\partial \omega_i} + \frac{\lambda}{N}\omega_i$$

Por tanto el peso $\omega_i$ se actualiza de la siguiente manera

$$\omega_i = \omega_i - \alpha\frac{\partial loss_{L2}}{\partial \omega_i} = \omega_i - \frac{\partial loss}{\partial \omega_i} - \frac{\lambda}{N}\omega_i$$

Es decir, en redes en las que un peso $\omega_i$ sea muy grande lo estamos reduciendo gracias al término $-\frac{\lambda}{N}\omega_i$. Por tanto, estamos reduciendo la complegidad de la red, ya que estamos reduciendo algunos pesos.

Hay que recordar, que los pesos simbolizan la union entre dos neuronas, por lo que reduciendo el valor de un peso, estamos reduciendo el enlace entre esas dos neuronas, hasta tal punto que si se reduce mucho podría equivaler a eliminar la unión entre esas neuronas.

Por lo tras varias épocas de entrenamiento, pasamos de una red grande, con muchas conexiones, a una red más simple, en la que algunas conexiones se han roto

¿Cómo se hace esto en Pytorch? Hasta ahora solo hemos visto el optimizador del [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD), así que vamos a ver cómo hacerlo para dicho optimizador, ya que para el resto es similar

Si recordamos, cada vez que hemos usado este optimizador lo hemos hecho así

```python
torch.optim.SGD(model.parameters(), lr=LR)
```

Es decir, le estamos pasando los parámetros del modelo y el learning rate, y cuando llamamos a su método `step()`

```python
optimizer.step()
```

Realiza la actualización de los parámetros de la manera que habíamos visto hasta ahora

Pero cuando se crea el optimizador pasándole el parámetro `weight_decay` realiza la regularización L2. En realidad siempre la está haciendo, lo que pasa es que por defecto, si no indicamos nada, `weight_decay = 0`, lo que supone $\lambda = 0$. De modo que a la hora de definir el optimizador tenemos que pasarle un valor a `weight_decay`, que es lo mismo que definir el valor de $\lambda$

```python
torch.optim.SGD(model.parameters(), lr=LR, weight_decay=X)
```

## Dropout

Supongamos que tenemos la siguiente red neuronal

![Dropout, red completa](Imagenes/Dropout_redCompleta.png)

Y ahora supongamos que lanzamos una moneda al aire para decidir si eliminamos cada una de las neuronas, es decir, cada neurona tiene una probabilidad de 0.5 de ser eliminada. Tras el lanzamiento de la moneda para cada neurona obtenemos que debemos eliminar las siguientes neuronas.

![Dropout, neuronas eliminadas](Imagenes/Dropout_NeuronasEliminadas.png)

Eliminar dichas neuronas también supone eliminar sus conexiones, por lo que nos queda la siguiente red neuronal

![Dropout, neuronas eliminadas](Imagenes/Dropout_redSimplificada.png)

Como vemos, lo que se ha hecho es simplificar la red

Este ha sido un ejemplo, pero lo que se hace en realidad es establecer una probabilidad de que se eliminen neuronas en cada capa. Dicha probabilidad no tiene que ser la misma para todas las capas.

De manera que durante el entrenamiento, en cada época, habrá unas neuronas eliminadas y otras no. En la siguiente época serán otras neuronas. Por lo que se va simplificando la red en cada época de una manera distinta

Esto hace que cada neurona, no se pueda fiar al 100% de lo que le llega de las anteriores neuronas, ya que en alguna época pueden desaparecer.

Esto solo se hace durante el entrenamiento, no durante el funcionamiento real de la red, o también llamado, durante la inferencia de la red. Es por esto, que cuando creabamos las funciones de validación poníamos el modelo en modo evaluación (`model.eval()`), haciendo esto se desactiva el dropout o cualquier otro modo de regularización. Mientras que en la función de entrenamiento, como poníamos el modelo en modo entrenamiento (`model.train()`), las técnicas de regularización se vuelven a activar

¿Como se implementa esto en python? Al igual que cuando se crea una red neuronal, se definen las capas y luego en la función `forward()` se conecta, con el dropout se hace igual, se definen los distintos dropouts para cada capa y después en la función `forward()` se conactan

Para crear cada capa de dropout se hace mediante `torch.nn.Dropout(p)` donde `p` es la probabilidad de eliminar neuronas

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_layers=[100, 50, 20]):
        super().__init__()
        self.layer1 = torch.nn.Linear(num_inputs, hidden_layers[0]),
        self.activation1 = torch.nn.Sigmoid()
        self.drop1 = torch.nn.Dropout(0.25)
        self.layer2 = torch.nn.Linear(hidden_layers[0], hidden_layers[1])
        self.activation2 = torch.nn.Sigmoid()
        self.drop2 = torch.nn.Dropout(0.5)
        self.layer3 = torch.nn.Linear(hidden_layers[1], hidden_layers[2])
        self.activation3 = torch.nn.Sigmoid()
        self.drop3 = torch.nn.Dropout(0.35)
        self.layer4 = torch.nn.Linear(hidden_layers[2], num_outputs)

    def forward(self, x):
        logits = self.layer1(x)
        logits = self.activation1(logits)
        logits = self.drop1(logits)
        logits = self.layer2(logits)
        logits = self.activation2(logits)
        logits = self.drop2(logits)
        logits = self.layer3(logits)
        logits = self.activation3(logits)
        logits = self.drop3(logits)
        logits = self.layer4(logits)
        return logits
```

## Data augmentation

Otra manera de evitar el sobreentrenamiento es añadir más datos, porque cuantos más datos se tenga, más dificil es para la red aprenderse todos y particularizar para ellos. Pero como generalmente esto no es posible, lo que se hace es transformar los que ya se tienen y metérselos a la red como si fuesen nuevos.

Esto lo veremos más en detalle en la parte de rededs convolucionales, pero a continuación se muestra un ejemplo

![data augmentation](Imagenes/data_augmentation.png)

## Early stopping

Otra forma de detectar y evitar el sobreentrenamiento es mediante el early stopping, o finalización temprana. Esto consiste en ir monitorizando el error en el conjunto de entrenamiento y de validación, ambos deberían ir disminuyendo, pero en el momento en el que el error del conjunto de entrenamiento sigue bajando, pero el del conjunto de validación comienza a subir es que estamos entrando en el sobreentrenamiento y debemos parar.

![early stopping](../01%20Introduccion%20a%20las%20redes%20neuronales/Imagenes/evolucion_error_train_test.png)

Esto sucede porque el modelo está empezando a aprenderse los datos de entrenamiento y no es capaz de generalizar para nuevos datos